In [4]:
import minsearch
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)    

In [6]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
    )

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [11]:
boost = {'question': 3.0, 'section': 0.6}

results = index.search(
    query=q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [12]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [13]:
import ollama

In [23]:
response = ollama.chat(
        model="llama3",
        messages=[{"role": "user", "content": q}]
    )

In [30]:
response['message']['content']

"The enthusiasm is palpable!\n\nWhile it's great that you're interested in the course, the answer to this question depends on the specific course and institution offering it.\n\nTypically, once a course has started, it may not be possible to enroll mid-semester. This is because:\n\n1. The course structure and syllabus have already been established.\n2. Students have likely already attended some classes, and it would be challenging for you to catch up on material that's already been covered.\n3. The instructor might have specific expectations and requirements for students who have already started the course.\n\nHowever, it's not always a hard no. Some courses or institutions might allow late enrollment under certain circumstances, such as:\n\n1. If there are still seats available (which is unlikely if the course has already started).\n2. If you can demonstrate that your interest in the course was genuine and you've been prevented from enrolling earlier due to unforeseen circumstances (e

In [31]:
prompt_template = """
You are a course teaching assistant. Answer the Question based on the Context.
Use only the facts from the Context when answering the Question.
If the Context does not contain enough information to answer the Question, respond with "I don't have enough information to answer the question."

Question: {question}

Context: {context}

"""

In [32]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [36]:
prompt = prompt_template.format(question=q, context=context).strip()

In [40]:
response = ollama.chat(
        model="llama3",
        messages=[{"role": "user", "content": prompt}]
    )

print(response['message']['content'])

"I don't have enough information to answer the question based on the provided Context. However, from a general understanding of course enrollment policies, if the context had mentioned that materials would be available after course completion or similar, one might infer eligibility for post-start enrollment in a self-paced mode. But since there is no specific information regarding enrolling after the start date in this context, I cannot provide an accurate answer."

In [39]:
def search(query):
    
    boost = {'question': 3.0, 'section': 0.5}
    
    results = index.search(
        query=query,
        filter_dict={"course": 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [42]:
def build_prompt(query, search_results):
    prompt_template = """
You are a course teaching assistant. Answer the Question based on the Context.
Use only the facts from the Context when answering the Question.
If the Context does not contain enough information to answer the Question, respond with "I don't have enough information to answer the question."

Question: {question}

Context: {context}

"""

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [65]:
def llm(prompt):
    response = ollama.chat(
        model="llama3",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response['message']['content']

In [59]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    
    return answer

In [66]:
query = 'how do I run kafka with python?'

answer = rag(query)

In [67]:
print(answer)

To run Kafka with Python, you can install `confluent-kafka` using pip:

```
pip install confluent-kafka
```

or using conda:

```
conda install conda-forge::python-confluent-kafka
```

Additionally, you may also need to install `fastavro`:

```
pip install fastavro
```


In [68]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [69]:
from elasticsearch import Elasticsearch

In [70]:
es_client = Elasticsearch('http://localhost:9200')

In [71]:
es_client.info()

ObjectApiResponse({'name': 'ca2f3f2ebd4b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dwMTbZ-GQ5CdFaGuz6zIWg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [73]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [74]:
from tqdm.auto import tqdm

In [75]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)
    

  0%|          | 0/948 [00:00<?, ?it/s]

In [109]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_doc = []
    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])
        
    return result_doc    

In [111]:
query = 'how do I run kafka?'

In [112]:
elastic_search(query=query)

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repositor